In [2]:
#we use the mnist dataset once again
import numpy as np
import matplotlib.pyplot as plt 
np.set_printoptions(threshold=np.inf)
data = np.load('mnist.npz')
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
#we normalize the data




In [3]:
#selecting class 0,1,2
class0 = x_train[y_train == 0]
class1 = x_train[y_train == 1]
class2 = x_train[y_train == 2]
print(class0.shape)
print(class1.shape)
print(class2.shape)
Ylabels = [0]*class0.shape[0] + [1]*class1.shape[0] + [2]*class2.shape[0]
Ylabels = np.array(Ylabels)
print(Ylabels.shape)
concat_classes = np.concatenate((class0,class1,class2), axis= 0)

print(concat_classes.shape)

(5923, 784)
(6742, 784)
(5958, 784)
(18623,)
(18623, 784)


In [4]:
def GiniIndex(leftlables, rightlables):
    left0count = np.sum(leftlables == 0)
    left1count = np.sum(leftlables == 1)
    left2count = np.sum(leftlables == 2)
    right0count = np.sum(rightlables == 0)
    right1count = np.sum(rightlables == 1)
    right2count = np.sum(rightlables == 2)
    lefttotal = left0count + left1count + left2count
    righttotal = right0count + right1count + right2count
    giniLeft = 1 - (left0count/lefttotal)**2 - (left1count/lefttotal)**2 - (left2count/lefttotal)**2
    giniRight = 1 - (right0count/righttotal)**2 - (right1count/righttotal)**2 - (right2count/righttotal)**2
    giniIndex = (lefttotal/(lefttotal + righttotal))*giniLeft + (righttotal/(lefttotal + righttotal))*giniRight
    return giniIndex
    

In [5]:
def makeSplit(Y,mutually_indexed_Ylabels, midpoints_per_dimension, dimension):
    leftlables = []
    rightlables = []
    left = []
    right = []
    for index, sample in enumerate(Y):
        if sample[dimension] < midpoints_per_dimension[dimension]:
            left.append(sample)
            leftlables.append(mutually_indexed_Ylabels[index])
        else:
            right.append(sample)
            rightlables.append(mutually_indexed_Ylabels[index])
    return np.array(left), np.array(right), np.array(leftlables), np.array(rightlables)


In [6]:
def bestSplit(Y,Ylabels, midpoints_per_dimension):
    gini = np.inf
    best_split = None
    for i in range(10):
        left, right, leftlables, rightlables = makeSplit(Y,Ylabels, midpoints_per_dimension, i)
        giniLoop = GiniIndex(leftlables, rightlables)
        if giniLoop < gini:
            gini = giniLoop
            best_split = (i,gini,left,right,leftlables,rightlables)
    return best_split

In [7]:
def predict(sample, firstmean, rightsplitmean):
    if(sample[0] < firstmean[0]):
        return 0
    else:
        if(sample[0] < rightsplitmean[0]):
            return 2
        else:
            return 1

    

In [8]:
def getPCA_Data(data,mean):
    data = data - mean
    cov = data.T @ data/18622
    eigenvalues,eigenvectors = np.linalg.eig(cov)
    idx = np.argsort(eigenvalues)[::-1]
    U = eigenvectors[:,idx]
    U = np.real(U)
    U = U[:,:10]
    Y = U.T @ data.T
    Y = Y.T
    Y = np.real(Y/255)
    return Y, U

In [9]:
def returnDictWithNumberofClasses(Ylabels):
    class_count = {0:np.sum(Ylabels == 0),1:np.sum(Ylabels == 1),2:np.sum(Ylabels == 2)}
    return class_count

In [10]:
def getBaggedAndReducedData(x_test,x_testLabels):
    indices = np.random.choice(18623, 18623, replace = True)
    baggedData = x_test[indices]
    baggedLabels = x_testLabels[indices]
    datamean = np.mean(baggedData, axis = 0)
    Y,U = getPCA_Data(baggedData,datamean)
    return Y,U,baggedLabels,datamean
    #apply the same transformation as before


In [11]:
#we write the part1 code in a function
def trainTree(Y,Ylabels,mean_vector):
    midpoints_per_dimension = [mean_vector[i] for i in range(10)]
    best_split = bestSplit(Y,Ylabels,midpoints_per_dimension)

    newleft = best_split[2]
    newleftlabels = best_split[4]
    class_count_firstLeft = returnDictWithNumberofClasses(newleftlabels)
    newLeftMean = np.mean(newleft, axis = 0)
    newLeftMidpoints = [newLeftMean[i] for i in range(10)]

    newright = best_split[3]
    newrightlabels = best_split[5]
    newRightMean = np.mean(newright, axis = 0)
    newRightMidpoints = [newRightMean[i] for i in range(10)]

    newLeftBestSplit = bestSplit(newleft,newleftlabels,newLeftMidpoints)
    newRightBestSplit = bestSplit(newright,newrightlabels,newRightMidpoints)
    
    newBestSplit = None
    if newLeftBestSplit[1] < newRightBestSplit[1]:
        newBestSplit = newLeftBestSplit
    else:
        newBestSplit = newRightBestSplit
    class_count_secondLeft = returnDictWithNumberofClasses(newBestSplit[4])
    class_count_secondRight = returnDictWithNumberofClasses(newBestSplit[5])
    print(best_split[0],best_split[1], newBestSplit[0],newBestSplit[1])
    return best_split, newBestSplit, class_count_firstLeft, class_count_secondLeft, class_count_secondRight

    

In [12]:
def predictArbitrary(sample, firstmean, rightsplitmean, class_count_firstLeft, class_count_secondLeft, class_count_secondRight):
    if(sample[0] < firstmean[0]):
        return max(class_count_firstLeft, key = class_count_firstLeft.get)
    else:
        if(sample[0] < rightsplitmean[0]):
            return max(class_count_secondLeft, key = class_count_secondLeft.get)
        else:
            return max(class_count_secondRight, key = class_count_secondRight.get)

In [13]:
def makeTest(test_concat_classes,mean_vector,U):
    test_concat_classes_func = test_concat_classes - mean_vector
    Y = U.T @ test_concat_classes_func.T
    Y = Y.T
    Y = np.real(Y/255)
    return Y

In [14]:
mean_highdim = np.mean(concat_classes, axis = 0)
Y,U = getPCA_Data(concat_classes,mean_highdim)
print(Y.shape)
mean_vector = np.mean(Y, axis = 0)
best_split, newBestSplit, class_count_first, class_count_secondLeft, class_count_secondRight = trainTree(Y,Ylabels,mean_vector)
print(best_split[0],best_split[1], newBestSplit[0],newBestSplit[1])
print(class_count_first, class_count_secondLeft, class_count_secondRight)   

(18623, 10)
0 0.44158414128431744 0 0.1508673520197507
0 0.44158414128431744 0 0.1508673520197507
{0: 5844, 1: 10, 2: 2768} {0: 79, 1: 661, 2: 3041} {0: 0, 1: 6071, 2: 149}


In [15]:
test_class0 = x_test[y_test == 0]
test_class1 = x_test[y_test == 1]
test_class2 = x_test[y_test == 2]
print(test_class0.shape)
print(test_class1.shape)
print(test_class2.shape)
testYlabels = [0]*test_class0.shape[0] + [1]*test_class1.shape[0] + [2]*test_class2.shape[0]
testYlabels = np.array(testYlabels)
print(testYlabels.shape)
test_concat_classes = np.concatenate((test_class0,test_class1,test_class2), axis= 0)

(980, 784)
(1135, 784)
(1032, 784)
(3147,)


In [16]:
Y_test = makeTest(test_concat_classes,mean_highdim,U)
print(Y_test.shape)

(3147, 10)


In [17]:
predictions = []
rightSplitmean = np.mean(best_split[3], axis = 0)
for sample in Y_test:
    predictions.append(predictArbitrary(sample,mean_vector,rightSplitmean,class_count_first, class_count_secondLeft, class_count_secondRight))
predictions = np.array(predictions)
print(predictions)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [18]:
#we calculate the accuracy
accuracy = np.sum(predictions == testYlabels)/len(predictions)
print(accuracy)

0.8090244677470607


THE FUNCTIONS ARE CORRECT. WE NEED TO IMPLEMENT BAGGING

In [19]:
#we now implement the bagging for 5 trees
Dis = []
Di_U = []
Di_labels = []
Di_mean = []
for i in range(5):
    Y,U,baggedLabels,datamean = getBaggedAndReducedData(concat_classes,Ylabels)
    Di_U.append(U)
    Di_labels.append(baggedLabels)
    Di_mean.append(datamean)
    Dis.append(Y)
    print(Y.shape, U.shape, baggedLabels.shape, datamean.shape)

(18623, 10) (784, 10) (18623,) (784,)
(18623, 10) (784, 10) (18623,) (784,)
(18623, 10) (784, 10) (18623,) (784,)
(18623, 10) (784, 10) (18623,) (784,)
(18623, 10) (784, 10) (18623,) (784,)


In [20]:
#now we train the trees
trees = []
for i in range(5):
    best_split, newBestSplit, class_count_first, class_count_secondLeft, class_count_secondRight = trainTree(Dis[i],Di_labels[i],Di_mean[i])
    trees.append((best_split, newBestSplit, class_count_first, class_count_secondLeft, class_count_secondRight)) 

    

0 0.4442169460136429 0 0.1530036275871741
0 0.4430221062394115 0 0.14882637350713349
0 0.4458166656309114 0 0.1597580383695303
0 0.4412952767188004 0 0.1583738161130442
0 0.4396144307106541 0 0.15039190857364887


In [21]:
Y_test_arr = []
for i in range(5):
    Y_test_temp = makeTest(test_concat_classes,Di_mean[i],Di_U[i])
    Y_test_arr.append(Y_test_temp)
    print(Y_test_temp.shape)

(3147, 10)
(3147, 10)
(3147, 10)
(3147, 10)
(3147, 10)


In [22]:
#we now make the predictions
predictions_arr = []
for i in range(5):
    rightSplitmean = np.mean(trees[i][0][3], axis = 0)
    pred_loop = []
    for sample in Y_test_arr[i]:
        pred_loop.append(predictArbitrary(sample,Di_mean[i],rightSplitmean,trees[i][2], trees[i][3], trees[i][4]))
    predictions_arr.append(np.array(pred_loop))
    print(predictions_arr[i].shape)

    

(3147,)


(3147,)
(3147,)
(3147,)
(3147,)


In [23]:
#we now conduct the majority vote by taking the mode of the predictions across the 5 trees
final_predictions = []
for i in range(len(predictions_arr[0])):
    pred_count = {0:0,1:0,2:0}
    for j in range(5):
        pred_count[predictions_arr[j][i]] += 1
    final_predictions.append(max(pred_count, key = pred_count.get))
    
final_predictions = np.array(final_predictions)
print(final_predictions.shape)

#we calculate the accuracy
accuracy = np.sum(final_predictions == testYlabels)/len(final_predictions)
print(accuracy)
    

(3147,)
0.8068001271051796


In [24]:
#class wise accuracy
class0_accuracy = np.sum(final_predictions[testYlabels == 0] == 0)/np.sum(testYlabels == 0)
class1_accuracy = np.sum(final_predictions[testYlabels == 1] == 1)/np.sum(testYlabels == 1)
class2_accuracy = np.sum(final_predictions[testYlabels == 2] == 2)/np.sum(testYlabels == 2)
print(class0_accuracy)
print(class1_accuracy)
print(class2_accuracy)


0.9908163265306122
0.9039647577092511
0.5251937984496124
